In [14]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FPGrowthExample").getOrCreate()
df = spark.read.csv("dulieugiaodich.csv", header=True, inferSchema=True)
df.show(100)


In [20]:
import pandas as pd
import pyfpgrowth

# Đọc dữ liệu từ tệp CSV vào một DataFrame
data = pd.read_csv("dulieugiaodich.csv")

# Chọn các cột cần thiết (Order ID và Product)
selected_data = data[["Order ID", "Product"]]

# Chuyển dữ liệu thành một danh sách dữ liệu (list of lists)
transactions = []
for order_id, group in selected_data.groupby("Order ID"):
    products = list(group["Product"])
    transactions.append([order_id] + products)

# Tạo tập dữ liệu để thực hiện FP-Growth
transactions_set = [set(record[1:]) for record in transactions]

# Thực hiện FP-Growth để đếm số lần xuất hiện của mỗi sản phẩm
patterns = pyfpgrowth.find_frequent_patterns(transactions_set, 1)  # Số 1 ở đây có thể được thay đổi tùy ý

# In kết quả
for pattern, support in patterns.items():
    print(f"Product: {pattern[0]}, Product: {pattern[1:]}, Support: {support}")


In [26]:
import pandas as pd
import pyfpgrowth

# Đọc dữ liệu từ tệp CSV vào một DataFrame
data = pd.read_csv("dulieugiaodich.csv")

# Tạo bảng tần suất xuất hiện của các sản phẩm
product_counts = data["Product"].value_counts().head(10)
print(product_counts)

# Chọn top 10 sản phẩm phổ biến nhất
top_products = product_counts.index.tolist()
print(top_products)

# # Lọc dữ liệu để chỉ bao gồm top 10 sản phẩm
filtered_data = data[data["Product"].isin(top_products)][["Order ID", "Product"]]
print(filtered_data)

# # Chuyển dữ liệu thành một danh sách dữ liệu (list of lists)
transactions = []
for order_id, group in filtered_data.groupby("Order ID"):
    products = list(group["Product"])
    transactions.append([order_id] + products)
# print(transactions)
# # Tạo tập dữ liệu để thực hiện FP-Growth
transactions_set = [set(record[1:]) for record in transactions]
# print(transactions_set)
# # Thực hiện FP-Growth để đếm số lần xuất hiện của mỗi sản phẩm
patterns = pyfpgrowth.find_frequent_patterns(transactions_set, 1)  # Số 1 ở đây có thể được thay đổi tùy ý
print(patterns)
# # Tạo DataFrame để trình bày dự đoán kết hợp sản phẩm với phần trăm
print('\nphan tram san pham se duoc chon trong tuong lai')
results = []
total_transactions = len(transactions_set)

for pattern, support in patterns.items():
    if len(pattern) > 1:
        order_id = pattern[0]
        combined_products = ", ".join(pattern[1:])
        percentage = (support / total_transactions) * 100
        results.append([order_id, combined_products, support, f"{percentage:.2f}%"])

df = pd.DataFrame(results, columns=["Order ID", "Combined Products", "Support", "Percentage"])
 
# In DataFrame
print(df.head(10))



In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Khởi tạo một phiên Spark
spark = SparkSession.builder.appName("MaxProductContact").getOrCreate()

# Đọc dữ liệu từ tệp CSV
df = spark.read.csv("dulieugiaodich.csv", header=True, inferSchema=True)

# Gom nhóm dữ liệu theo 'Contact Name' và đếm số lượng sản phẩm cho mỗi nhóm
grouped = df.groupBy('Contact Name').agg({"Product": "count"})

# Sắp xếp kết quả theo số lượng sản phẩm giảm dần
sorted_grouped = grouped.sort(col("count(Product)").desc())
# Lấy `Contact Name` của nhóm đầu tiên (có số lượng sản phẩm nhiều nhất)
most_product_contact = sorted_grouped.first()['Contact Name']

print("Contact Name có số lượng sản phẩm nhiều nhất là:", most_product_contact)
# Dừng phiên Spark
spark.stop()


Contact Name có số lượng sản phẩm nhiều nhất là: Leonard Kelly


In [ ]:
#tap trung phan tich du liệu của người mua sản phâm nhiều nhất là Leonard Kelly
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import col
# Khởi tạo phiên Spark
spark = SparkSession.builder.appName("AmazonFPAnalysis").getOrCreate()
#loc ra 10 san pham pho bien nhat trong cua hang thong qua du lieu da lam 
data = [(1, ["ContactMatcher","Storage","Site Analytics"]),
        (2, ["SaaS Connector Pack"]),
        (3, ["SaaS Connector Pack","Support","ContactMatcher"]),
        (4, ["Storage"]),
        (5, ["Site Analytics","ContactMatcher","Support","Big Ol Database"]),
        (6, ["ContactMatcher","Marketing Suite","ChatBot Plugin","Data Smasher"]),
        (7, ["OneView","Data Smasher"]),
        (8, ["Site Analytics","OneView"]),
        (9, ["ContactMatcher","ChatBot Plugin","Support","Data Smasher","FinanceHub","Marketing Suite","OneView"]),
        (10, ["ContactMatcher","SaaS Connector Pack"]),
        (11, ["SaaS Connector Pack","ContactMatcher"])]

columns = ["Order ID", "Product"]

df = spark.createDataFrame(data, columns)

df.show()

# Sử dụng FP-Growth để tìm các tập mục thường xuyên
fp_growth = FPGrowth(itemsCol="Product", minSupport=0.1, minConfidence=0.1)
model = fp_growth.fit(df)

# Hiển thị tập hợp các mục thường xuyên
frequent_itemsets = model.freqItemsets
# frequent_itemsets.show()
frequent_itemsets = frequent_itemsets.withColumn("items", frequent_itemsets["items"].cast("string"))

frequent_itemsets.show(truncate=False)

# Hiển thị quy tắc liên kết
association_rules = model.associationRules
# association_rules.show()
association_rules = association_rules.withColumnRenamed("antecedent", "X")
association_rules = association_rules.withColumnRenamed("consequent", "Y")

association_rules = association_rules.withColumn("X", col("X").cast("string"))
association_rules = association_rules.withColumn("Y", col("Y").cast("string"))

association_rules.show(truncate=False)
